## Анализ олимпиад

## Описание данных

`ID` - порядковый номер участника в таблице

`Пол` - пол учасника (женский, мужской)

`Регион, место нахождения` - регион написания участником олимпиады (?)

`Класс` - класс, за который участник писал олимпиаду

`Статус` - статус участника (победитель, призёр)

`Предмет` - предмет олимпиады

## Цель работы

Проверить на достоверность основные стереотипы, связанные с проведением Всероссийской олимпиады школьников

## Предобработка данных

### Импортирование библиотек

In [2]:
import pandas as pd
from scipy import stats as st
import math as mth
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

### Знакомство с данными

In [3]:
data = pd.read_excel("C:/Users/larag/Downloads/Telegram Desktop/Данные_02.xlsx")
data.head()

,ID,Пол,"Регион, место нахождения",Класс,Статус,Предмет,Область изучения,Федеральный округ
0,1,Женский,Москва,8,Победитель,Английский,Гуманитарные,Центральный федеральный округ
1,2,Мужской,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ
2,3,Женский,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ
3,4,Мужской,Московская область,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ
4,5,Мужской,Магаданская область,9,Победитель,Английский,Гуманитарные,Дальневосточный федеральный округ


Названия столбцов необходимо привести к соответствующему формату

In [4]:
data.columns=['id', 'sex', 'region', 'grade', 'status', 'subject', 'field', 'district']
data.columns

Index(['id', 'sex', 'region', 'grade', 'status', 'subject', 'field',
       'district'],
      dtype='object')

Названия столбцов приведены к соответствующему формату

### Проверка значений

#### Столбец `status`

In [5]:
data['status'].value_counts()

Призер        2259
Победитель     488
Name: status, dtype: int64

В таблице присутствует 24 значения с неверным написанием статуса ("Победителиь"). Исправлю их

In [6]:
data['status'] = data['status'].replace('Победителиь', 'Победитель')
data['status'].value_counts()

Призер        2259
Победитель     488
Name: status, dtype: int64

Значения исправлены

#### Столбец `sex`

In [7]:
data['sex'].value_counts()

Мужской    1797
Женский     950
Name: sex, dtype: int64

Неверных значений нет

#### Столбец `grade`

In [8]:
data['grade'].value_counts()

11        1063
10         887
9          671
8          102
7           18
1 курс       2
12           1
5            1
2 курс       1
6            1
Name: grade, dtype: int64

В таблице присутствуют значения класса 12, 1 курс и второй курс. Исходя из того, что в работе рассматривается олимпиада школьников России, удалю аномальные значения

In [9]:
data = data[~data['grade'].isin([12, "1 курс", "2 курс"])]
data['grade'].value_counts()

11    1063
10     887
9      671
8      102
7       18
5        1
6        1
Name: grade, dtype: int64

Аномальные значения удалены

#### Столбец `subject`

In [10]:
data['subject'].unique()

array(['Английский ', 'Астрономия', 'Биология', 'География',
       'Информатика', 'МХК', 'История', 'Литература', 'Математика',
       'Обществознание', 'ОБЖ', 'Право', 'Русский язык', 'Технология',
       'Физика', 'Физическая культура', 'Химия', 'Экология', 'Экономика'],
      dtype=object)

Обнаружено написание предмета Английский с пробелом и ошибочное написание столбца  Обществознание

In [11]:
data['subject'] = data['subject'].apply(str.strip).replace('Обществознани', 'Обществознание')
data['subject'].unique()

array(['Английский', 'Астрономия', 'Биология', 'География', 'Информатика',
       'МХК', 'История', 'Литература', 'Математика', 'Обществознание',
       'ОБЖ', 'Право', 'Русский язык', 'Технология', 'Физика',
       'Физическая культура', 'Химия', 'Экология', 'Экономика'],
      dtype=object)

Значения исправлены

In [12]:
data['region'].unique()

array(['Москва', 'Московская область', 'Магаданская область',
       'Пензенская область', 'Санкт-Петербург', 'Калужская область',
       'Республика Татарстан', 'Удмуртская Республика',
       'Челябинская область', 'Ямало-Ненецкий автономный округ',
       'Приморский край', 'Республика Башкортостан', 'Брянская область',
       'Владимирская область', 'Республика Крым', 'Мурманская область',
       'Красноярский край', 'Сахалинская область', 'Краснодарский край',
       'Ивановская область', 'Пермский край', 'Республика Дагестан',
       'Ростовская область', 'Чувашская Республика - Чувашия',
       'Свердловская область', 'Томская область', 'Вологодская область',
       'Камчатский край', 'Новосибирская область', 'Республика Мордовия',
       'Калининградская область', 'Архангельская область',
       'Ставропольский край', 'Ленинградская область',
       'Тюменская область', 'Кировская область', 'Республика Коми',
       'Челябинская область ', 'Омская область', 'Оренбургская област

Присутствуют дубликаты из-за пробелов в конце строк

In [13]:
print(f'Количетво уникальных значений столбца region до удаления дубликатов {len(data["region"].unique())}')
data['region'] = data['region'].apply(str.strip)
print(f'Количетво уникальных значений столбца region после удаления дубликатов {len(data["region"].unique())}')

Количетво уникальных значений столбца region до удаления дубликатов 81
Количетво уникальных значений столбца region после удаления дубликатов 76


Дубликаты, связанные с пробелами в конце строк удалены

*Промежуточный вывод*

Дубликаты и ошибочные значения исправлены

### Оценка типов данных

In [14]:
data.dtypes

id           int64
sex         object
region      object
grade       object
status      object
subject     object
field       object
district    object
dtype: object

Типы данных соответствуют действительности

### Добавление новых столбцов

Добавлю столбец `region` - отдалённость региона от Москвы (по прямой)

In [15]:
def km(x):
    if x == 'Москва':
        return 0
    if x == 'Московская область':
        return 43
    if x == 'Магаданская область':
        return 5306
    if x == 'Пензенская область':
        return 555
    if x == 'Санкт-Петербург':
        return 639
    if x == 'Калужская область':
        return 163
    if x == 'Республика Татарстан':
        return 523
    if x == 'Челябинская область':
        return 1495
    if x == 'Ямало-Ненецкий автономный округ':
        return 2317
    if x == 'Приморский край':
        return 6423
    if x == 'Республика Башкортостан':
        return 1156
    if x == 'Брянская область':
        return 349
    if x == 'Владимирская область':
        return 178
    if x == 'Республика Крым':
        return 949
    if x == 'Мурманская область':
        return 925
    if x == 'Красноярский край':
        return 3355
    if x == 'Сахалинская область':
        return 4132
    if x == 'Краснодарский край':
        return 1195
    if x == 'Ивановская область':
        return 249
    if x == 'Пермский край':
        return 1155
    if x == 'Республика Дагестан':
        return 1586
    if x == 'Ростовская область':
        return 957
    if x == 'Чувашская Республика - Чувашия':
        return 601
    if x == 'Свердловская область':
        return 1415
    if x == 'Томская область':
        return 2875
    if x == 'Вологодская область':
        return 408
    if x == 'Камчатский край':
        return 3273
    if x == 'Новосибирская область':
        return 2809
    if x == 'Республика Мордовия':
        return 513
    if x == 'Калининградская область':
        return 1088
    if x == 'Архангельская область':
        return 992
    if x == 'Ставропольский край':
        return 1229
    if x == 'Ленинградская область':
        return 1055
    if x == 'Тюменская область':
        return 1709
    if x == 'Кировская область':
        return 789
    if x == 'Республика Коми':
        return 1003
    if x == 'Омская область':
        return 2235
    if x == 'Оренбургская область':
        return 1226
    if x == 'Республика Бурятия':
        return 4418
    if x == 'Смоленская область':
        return 368
    if x == 'Самарская область':
        return 856
    if x == 'Тульская область':
        return 173
    if x == 'Псковская область':
        return 610
    if x == 'Республика Адыгея':
        return 1251
    if x == 'Нижегородская область':
        return 401
    if x == 'Республика Саха (Якутия) ':
        return 4880
    if x == 'Хабаровский край':
        return 6135
    if x == 'Кемеровская область – Кузбасс':
        return 2984
    if x == 'Ханты-Мансийский автономный округ – Югра':
        return 1898
    if x == 'Рязанская область':
        return 184
    if x == 'Алтайский край':
        return 2929
    if x == 'Белгородская область':
        return 575
    if x == 'Тверская область':
        return 163
    if x == 'Костромская область':
        return 302
    if x == 'Севастополь':
        return 1274
    if x == 'Ярославская область':
        return 249
    if x == 'Новгородская область':
        return 491
    if x == 'Саратовская область':
        return 723
    if x == 'Тамбовская область':
        return 417
    if x == 'Иркутская область':
        return 4203
    if x == 'Орловская область':
        return 325
    if x == 'Республика Марий Эл':
        return 641
    if x == 'Республика Карелия':
        return 700
    if x == 'Луганская Народная Республика':
        return 807
    if x == 'Республика Алтай':
        return 913
    if x == 'Ульяновская область':
        return 704
    if x == 'Курганская область':
        return 1729
    if x == 'Липецкая область':
        return 371
    if x == 'Кабардино-Балкарская Республика':
        return 1428
    if x == 'Курская область':
        return 457
    if x == 'Воронежская область':
        return 465
    if x == 'Республика Калмыкия':
        return 1146
    if x == 'Ненецкий автономный округ':
        return 1542
    if x == 'Республика Хакасия':
        return 3372
    if x == 'Удмуртская Республика':
        return 968

In [16]:
data['distance'] = data['region'].apply(km)
data.head()

,id,sex,region,grade,status,subject,field,district,distance
0,1,Женский,Москва,8,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0
1,2,Мужской,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0
2,3,Женский,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0
3,4,Мужской,Московская область,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,43.0
4,5,Мужской,Магаданская область,9,Победитель,Английский,Гуманитарные,Дальневосточный федеральный округ,5306.0


Новый столбец добавлен

Добавлю столбец `status_id` - идентификатор статуса участника (1 - победитель, 0 - призёр)

In [17]:
def status_id(row):
    if row == 'Победитель':
        return 1
    return 0

In [18]:
data['status_id'] = data['status'].apply(status_id)
data.head()

,id,sex,region,grade,status,subject,field,district,distance,status_id
0,1,Женский,Москва,8,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0,1
1,2,Мужской,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0,1
2,3,Женский,Москва,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,0.0,1
3,4,Мужской,Московская область,9,Победитель,Английский,Гуманитарные,Центральный федеральный округ,43.0,1
4,5,Мужской,Магаданская область,9,Победитель,Английский,Гуманитарные,Дальневосточный федеральный округ,5306.0,1


Новый столбец добавлен

## Проверка значимости различия числа призёров и победителей по полу

*Нулевая гипотеза:* распределение нормальное

*Альтернативная гипотеза:* распределение не нормальное

In [19]:
def distrib(sample):

    alpha = 0.05

    results = st.shapiro(sample)
    p_value = results[1]

    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: распределение не нормально')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, распределение нормальное')

In [20]:
sex_winners = data.pivot_table(index='subject', columns='sex', values='id', aggfunc='count')
total = data.groupby('sex')['id'].count()

In [21]:
print('Проверка распределения количества победителей и призёров по предметов среди девушек')
distrib(sex_winners['Женский'])

print()

print('Проверка распределения количества победителей и призёров по предметов среди юношей')
distrib(sex_winners['Мужской'])

Проверка распределения количества победителей и призёров по предметов среди девушек
p-значение:  0.5202280879020691
Не получилось отвергнуть нулевую гипотезу, распределение нормальное

Проверка распределения количества победителей и призёров по предметов среди юношей
p-значение:  0.2625121772289276
Не получилось отвергнуть нулевую гипотезу, распределение нормальное


Распределение количества победителей и призёров по предметов среди обоих полов нормальное

In [35]:
def p_v(table, total):
    women_total = total['Женский']
    men_total = total['Мужской']
    alpha = 0.05
    
    for subject in table.index:
        women = table.loc[subject]['Женский']
        men = table.loc[subject]['Мужской']
        
        pw = women / women_total

        pm = men/men_total

        p_combined = (women + men) / (women_total + men_total)

        difference = pw - pm

        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/women_total + 1/men_total))

        distr = st.norm(0, 1) 

        print(f'Проверка предмета {subject}')
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        print('p-значение: ', p_value)

        if p_value < alpha:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
        print()

*Нулевая гипотеза:* Между долями нет статистически значимой разницы

*Альтернативная гипотеза:* Между долями есть статистически значимая разница

Результаты статистической проверки гипотезы о разнице долей успехов между женской и мужской группами на основе данных о количестве призёров и победителей всероссийской олимпиады школьников

In [23]:
p_v(sex_winners, total)

Проверка предмета Английский
p-значение:  0.00030214417560037177
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета Астрономия
p-значение:  6.842670869922074e-07
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета Биология
p-значение:  0.006720238080199836
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета География
p-значение:  1.7549846886844023e-07
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета Информатика
p-значение:  0.0
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета История
p-значение:  0.058256140844969995
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка предмета Литература
p-значение:  0.0
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка предмета МХК
p-значение:  1.6653345369377348e-14
Отвергаем нулевую гипотезу: между долями есть значимая разница

Проверка

Нет оснований считать доли побидителей и призёров по полу разными только при рассмотрении предметов физическая культура, технология, обществознания и история (4 из 19 предметов)

## Проверка зависимости количества победителей от отдалённости регионов от Москвы

In [33]:
corr_data = data.groupby('distance')['status_id'].sum()
corr_data = corr_data.reset_index()
corr_data = corr_data[corr_data['status_id'] != 0]
corr_data.columns=['Расстояние до Москвы', 'Количество победителей']
corr_data = corr_data.sort_values(by='Расстояние до Москвы')
corr_data.head()

,Расстояние до Москвы,Количество победителей
0,0.0,247
1,43.0,69
2,163.0,1
6,249.0,5
12,401.0,3


In [34]:
corr = corr_data['Расстояние до Москвы'].corr(corr_data['Количество победителей'])
corr

-0.2357095177242441

Между расстоянием города до Москвы и количеством победителей в нем нет зависимости

## Выводы

- Распределение количества победителей и призёров по предметов среди обоих полов нормальное

- Нет оснований считать доли побидителей и призёров по полу разными только при рассмотрении предметов физическая культура, технология, обществознания и история (4 из 19 предметов)

- Между расстоянием города до Москвы и количеством победителей в нем нет зависимости